In [ ]:
import param
import numpy as np
import pandas as pd


## TODO: this has been factored out and into neuropy.neuron_identities.NeuronIdentityAccessingMixin
class NeuronIdentityAccessingMixin:
    @property
    def neuron_ids(self):
        """ e.g. return np.array(active_epoch_placefields2D.cell_ids) """
        raise NotImplementedError
    
    def get_neuron_id_and_idx(self, neuron_i=None, neuron_id=None):
        assert (neuron_i is not None) or (neuron_id is not None), "You must specify either cell_i or cell_id, and the other will be returned"
        if neuron_i is not None:
            neuron_i = int(neuron_i)
            neuron_id = self.neuron_ids[neuron_i]
        elif neuron_id is not None:
            neuron_id = int(neuron_id)
            neuron_i = np.where(self.neuron_ids == neuron_id)[0].item()
        # print(f'cell_i: {cell_i}, cell_id: {cell_id}')
        return neuron_i, neuron_id
    
class HideShowSpikeRenderingMixin:
    def update_active_spikes(self, spike_opacity_mask):
        """ 
        Usage: 
            included_cell_ids = [48, 61]
            
            ipcDataExplorer.update_active_spikes(np.isin(ipcDataExplorer.active_session.spikes_df['aclu'], included_cell_ids)) # actives only the spikes that have aclu values (cell ids) in the included_cell_ids array.
        """
        assert np.shape(self.active_session.spikes_df['render_opacity']) == np.shape(spike_opacity_mask), "spike_opacity_mask must have one value for every spike in self.active_session.spikes_df, specifying its opacity"
        self.active_session.spikes_df['render_opacity'] = spike_opacity_mask
        self.update_spikes()
        
        
        
class HideShowPlacefieldsRenderingMixin(NeuronIdentityAccessingMixin, param.Parameterized):
    
    active_pf_idx_list = param.ListSelector(default=[3, 5], objects=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], precedence=0.5)
    # phase = param.Number(default=0, bounds=(0, np.pi))
    # frequency = param.Number(default=1, bounds=(0.1, 2))
    
    
    @param.depends('active_pf_idx_list')
    def interact_update_active_placefields(self):
        selected_placefield_indicies = self.active_pf_idx_list
        print(f'selected_placefield_indicies: {selected_placefield_indicies}')
        self.update_active_placefields(selected_placefield_indicies)
        
        
    def update_active_placefields(self, placefield_indicies):
        """ 
        Usage: 
            included_cell_ids = [48, 61]
            included_cell_INDEXES = [ipcDataExplorer.get_neuron_id_and_idx(cell_id=an_included_cell_ID)[0] for an_included_cell_ID in included_cell_ids] # get the indexes from the cellIDs
            ipcDataExplorer.update_active_placefields(included_cell_INDEXES) # actives only the placefields that have aclu values (cell ids) in the included_cell_ids array.
        """
        self._hide_all_tuning_curves() # hide all tuning curves to begin with (for a fresh slate)
        for a_pf_idx in placefield_indicies:
            self._show_tuning_curve(a_pf_idx)
        
    def _hide_all_tuning_curves(self):
        # Works to hide all turning curve plots:
        for aTuningCurveActor in self.plots['tuningCurvePlotActors']:
            aTuningCurveActor.SetVisibility(0)
            
    def _show_tuning_curve(self, show_index):
        # Works to show the specified tuning curve plots:
        self.plots['tuningCurvePlotActors'][show_index].SetVisibility(1)



class BaseClass(param.Parameterized):
    name = param.Parameter(default="Not editable", constant=True)
    isVisible = param.Boolean(True, doc="Whether the plot is visible")
    


class ExampleExtended(BaseClass):
    color                   = param.Color(default='#BBBBBB')
    dictionary              = param.Dict(default={"a": 2, "b": 9})
    select_string           = param.ObjectSelector(default="yellow", objects=["red", "yellow", "green"])
    select_fn               = param.ObjectSelector(default=list,objects=[list, set, dict])
    int_list                = param.ListSelector(default=[3, 5], objects=[1, 3, 5, 7, 9], precedence=0.5)

# checkbutton_group = pn.widgets.CheckButtonGroup(name='Check Button Group', value=[], options=pf_options_list_strings) # checkbutton_group.value 
# cross_selector = pn.widgets.CrossSelector(name='Active Placefields', value=[], options=pf_options_list_strings) # cross_selector.value

class SinglePlacefieldPlottingExtended(BaseClass):
    color = param.Color(default='#FF0000')
    spikesVisible = param.Boolean(True, doc="Whether the spikes are visible")
    extended_values_dictionary = param.Dict(default=dict())
    
    
# class SinglePlacefieldPlottingExtended(BaseClass):
#     color = param.Color(default='#FF0000')
#     spikesVisible = param.Boolean(True, doc="Whether the spikes are visible")
    
#     # dictionary              = param.Dict(default={"a": 2, "b": 9})
#     select_string           = param.ObjectSelector(default="yellow", objects=["red", "yellow", "green"])
#     select_fn               = param.ObjectSelector(default=list,objects=[list, set, dict])
#     # active_pf_idx_list      = param.ListSelector(default=[3, 5], objects=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], precedence=0.5)
    
#     def panel(self):
#         return pn.Row(
#             pn.Column(
#                 pn.Param(SinglePlacefieldPlottingExtended.param, name="SinglePlacefield", widgets= {
#                     'color': {'widget_type': pn.widgets.ColorPicker, 'name':'pf Color', 'value':'#99ef78', 'width': 50},
#                 })
#             )
#         )
    
    

In [ ]:
import panel as pn
from panel.viewable import Viewer

# .bk-root .bk-input {
#     padding: 0 0px;
# }

css = '''
.bk.no-widget-padding {
  padding: 0 0px;
}
''' # use via: css_classes=['no-widget-padding']
pn.extension(raw_css=[css])

In [ ]:
class EditableRange(Viewer):
    
    value = param.Range(doc="A numeric range.")
    
    width = param.Integer(default=300)
    
    def __init__(self, **params):
        self._start_input = pn.widgets.FloatInput()
        self._end_input = pn.widgets.FloatInput(align='end')
        super().__init__(**params)
        self._layout = pn.Row(self._start_input, self._end_input)
        self._sync_widgets()
    
    def __panel__(self):
        return self._layout
    
    @param.depends('value', 'width', watch=True)
    def _sync_widgets(self):
        self._start_input.name = self.name
        self._start_input.value = self.value[0]
        self._end_input.value = self.value[1]
        self._start_input.width = self.width//2
        self._end_input.width = self.width//2
        
    @param.depends('_start_input.value', '_end_input.value', watch=True)
    def _sync_params(self):
        self.value = (self._start_input.value, self._end_input.value)

range_widget = EditableRange(name='Range', value=(0, 10))

pn.Column(
    '## This is a custom widget',
    range_widget
)

In [ ]:
gspec = pn.GridSpec(sizing_mode='stretch_both', max_height=800)

gspec[0, :3] = pn.Spacer(background='#FF0000')
gspec[1:3, 0] = pn.Spacer(background='#0000FF')
gspec[1:3, 1:3] = fig
gspec[3:5, 0] = hv.Curve([1, 2, 3])
gspec[3:5, 1] = 'https://upload.wikimedia.org/wikipedia/commons/4/47/PNG_transparency_demonstration_1.png'
gspec[4:5, 2] = pn.Column(
    pn.widgets.FloatSlider(),
    pn.widgets.ColorPicker(),
    pn.widgets.Toggle(name='SpikesVisible'))

gspec

In [ ]:
test_single_pf_params = SinglePlacefieldPlottingExtended()

pn.Row(test_single_pf_params.param)
# SinglePlacefieldPlottingExtended.param

In [ ]:
def build_single_placefield_output_panel(name, color):
    # wgt_color_picker = pn.widgets.ColorPicker(value='#99ef78', width=60, max_width=50, height=10, margin=0, width_policy='fit', css_classes=['no-widget-padding'], sizing_mode='fixed', background='#99ef78')
    wgt_label_button = pn.widgets.Button(name='pf[i]', button_type='default', margin=0, height=20, sizing_mode='stretch_both', width_policy='min')
    wgt_color_picker = pn.widgets.ColorPicker(value='#99ef78', width=60, height=20, margin=0)
    wgt_toggle_visible = pn.widgets.Toggle(name='isVisible', margin=0)
    wgt_toggle_spikes = pn.widgets.Toggle(name='SpikesVisible', margin=0)
    # pn.Column(
    #     wgt_color_picker,
    #     wgt_toggle_visible, wgt_toggle_spikes,
    #     width=60, max_width=50, margin=0)

    gspec = pn.GridSpec(width=100, height=100, margin=0)
    # gspec = pn.GridSpec(sizing_mode='stretch_both', max_height=800)

    # gspec[0, :3] = pn.Spacer(background='#FF0000', margin=0)
    # gspec[1, :] = pn.Spacer(background='#0000FF', margin=0)
    # gspec[2, :] = pn.Spacer(background='#00F0FF', margin=0)

    # gspec[0, :3] = pn.widgets.StaticText(name='pf[i]')
    gspec[0, :3] = wgt_label_button
    gspec[1, :] = wgt_color_picker
    gspec[2, :] = pn.Row(wgt_toggle_visible, margin=0, background='red')
    gspec[3, :] = pn.Row(wgt_toggle_spikes, margin=0, background='green')

    # gspec[1:3, 0] = pn.Spacer(background='#0000FF')
    return gspec


build_single_placefield_output_panel()

In [ ]:
gspec = pn.GridSpec(width=100, height=100)

gspec[:,   0  ] = pn.Spacer(background='red',    margin=0)
gspec[0,   1:3] = pn.Spacer(background='green',  margin=0)
gspec[1,   2:4] = pn.Spacer(background='orange', margin=0)
gspec[2,   1:4] = pn.Spacer(background='blue',   margin=0)
gspec[0:1, 3:4] = pn.Spacer(background='purple', margin=0)

active_single_pf_panel = SinglePlacefieldPlottingExtended()

# gspec[:,   0  ] = pn.widgets.ColorPicker(value='#99ef78', width=50, margin=0, align=('start', 'end'), css_classes=['no-widget-padding'])
gspec[:,   0  ] = pn.widgets.ColorPicker(background='#99ef78', value='#99ef78', width=60, max_width=50, height=50, margin=0, width_policy='fit', css_classes=['no-widget-padding'], sizing_mode='fixed')


gspec

In [ ]:
# base = BaseClass()
active_single_pf_panel = SinglePlacefieldPlottingExtended()
active_single_pf_panel.panel()
# pn.Row(pn.Column(SinglePlacefieldPlottingExtended.param, width=40))

In [ ]:
active_pf_panel = ActivePlacefieldsPlotting(num_pfs=20)
# active_pf_panel
active_pf_panel.panel()

In [ ]:
# base = BaseClass()

# pn.Param(ActivePlacefieldsPlotting.param, widgets={
#     'active_pf_idx_list': {'widget_type': pn.widgets.CrossSelector, 'options': pf_options_list_strings},
#     # 'autocomplete_string': {'widget_type': pn.widgets.AutocompleteInput, 'placeholder': 'Find a color...'},
#     # 'select_number': pn.widgets.DiscretePlayer
#     }
# )

# pn.Row(ActivePlacefieldsPlotting.param, base.param)

# pn.Row(pn.Param(ActivePlacefieldsPlotting.param, name="Active Placefields", widgets={
#     'active_pf_idx_list': {'widget_type': pn.widgets.CrossSelector, 'name':'Active Placefields', 'options': pf_options_list_strings, 'height': 600},
#     # 'autocomplete_string': {'widget_type': pn.widgets.AutocompleteInput, 'placeholder': 'Find a color...'},
#     # 'select_number': pn.widgets.DiscretePlayer
#     }
# ))

class ActivePlacefieldsPlottingPanel(ActivePlacefieldsPlotting):
    
    def __init__(self, num_pfs=40, **params):
        super(ActivePlacefieldsPlottingPanel, self).__init__(num_pfs=num_pfs, **params)
        self.num_pfs = num_pfs
        # self.figure = figure(x_range=(-1, 1), y_range=(-1, 1))
        # self.renderer = self.figure.line(*self._get_coords())
    
    @staticmethod
    def build_pf_options_list(num_pfs=40):
        pf_options_list_ints = np.arange(num_pfs)
        pf_options_list_strings = [f'{i}' for i in pf_options_list_ints]
        return pf_options_list_ints, pf_options_list_strings
    
    # _on_hide_all_placefields = lambda x: print(f'_on_hide_all_placefields({x})')
    # _on_update_active_placefields = lambda x: print(f'_on_update_active_placefields({x})')
    
    def on_hide_all_placefields(self):
        print('on_hide_all_placefields()')
        # lambda x: print(f'_on_hide_all_placefields({x})')
    
    def on_update_active_placefields(self, updated_pf_indicies):
        print(f'on_update_active_placefields({updated_pf_indicies})')
        
    def btn_hide_all_callback(self, event):
        print('btn_hide_all_callback(...)')
        self.on_hide_all_placefields()
        
    def btn_update_active_placefields(self, event):
        print('btn_update_active_placefields(...)')
        self.on_update_active_placefields(self.cross_selector.value)
        
    def index_selection_changed_callback(self, *events):
        print(events)
        for event in events:
            if event.name == 'options':
                self.selections.object = 'Possible options: %s' % ', '.join(event.new)
            elif event.name == 'value':
                self.selected.object = 'Selected: %s' % ','.join(event.new)

    def panel(self):
        # Panel pane and widget objects:
        self.selections = pn.pane.Markdown(object='')
        self.selected = pn.pane.Markdown(object='')
        self.cross_selector = pn.widgets.CrossSelector(name='Active Placefields', value=[], options=['0', '1', '2'], height=600, width=200) # cross_selector.value

        # Action Buttons:
        self.button_hide_all = pn.widgets.Button(name='Hide All Placefields')
        self.button_hide_all.on_click(self.btn_hide_all_callback)
        self.button_update = pn.widgets.Button(name='Update Active Placefields', button_type='primary')
        self.button_update.on_click(self.btn_update_active_placefields)

        self.watcher = self.cross_selector.param.watch(self.index_selection_changed_callback, ['options', 'value'], onlychanged=False)
        # set initial
        active_new_pf_panel.set_initial(self.num_pfs, [0, 1, 5])
        
        return pn.Column(pn.Row(self.cross_selector, width=200, height=600),
                         pn.Spacer(width=200, height=10),
                         self.selections,
                         pn.Spacer(width=200, height=10),
                         self.selected,
                         pn.Spacer(width=200, height=20),
                         pn.Row(self.button_hide_all, self.button_update)
                        )
    
        # return pn.Row(pn.Column(self.cross_selector, width=200, height=600), self.selections, pn.Spacer(width=50, height=600), self.selected)

    def set_initial(self, num_pfs, selected_values):
        # set initial
        # options = ['A','B','C','D']
        # options = ['A','B','C','D']
        pf_options_list_ints, pf_options_list_strings = ActivePlacefieldsPlotting.build_pf_options_list(num_pfs)
        options = pf_options_list_strings
        selected_values = [str(an_item) for an_item in selected_values]
        # value=[]
        # self.cross_selector.param.set_param(options=dict(zip(options,options)), value=['D'])
        # self.cross_selector.param.set_param(options=dict(zip(options,options)), value=['D'])
        self.cross_selector.param.set_param(options=dict(zip(options, options)), value=selected_values)
        

active_new_pf_panel = ActivePlacefieldsPlottingPanel(num_pfs=40)
active_new_pf_panel.panel()

In [ ]:
active_new_pf_panel.set_initial(10, [0, 1, 5])

In [ ]:
markdown = pn.pane.Markdown("Some text")
text_input = pn.widgets.TextInput(value=markdown.object)

text_input.link(markdown, value='object')

pn.Row(text_input, markdown)

In [ ]:
m = pn.pane.Markdown("")
t = pn.widgets.TextInput()

def callback(target, event):
    target.object = event.new.upper() + '!!!'

t.link(m, callbacks={'value': callback})
t.value="Some text"

pn.Row(t, m)

In [ ]:
selections = pn.pane.Markdown(object='')
selected = pn.pane.Markdown(object='')
toggle = pn.widgets.ToggleGroup(options=['A', 'B'])

def callback(*events):
    print(events)
    for event in events:
        if event.name == 'options':
            selections.object = 'Possible options: %s' % ', '.join(event.new)
        elif event.name == 'value':
            selected.object = 'Selected: %s' % ','.join(event.new)
            
watcher = toggle.param.watch(callback, ['options', 'value'], onlychanged=False)

pn.Row(pn.Column(toggle, width=200, height=50), selections, pn.Spacer(width=50, height=50), selected)

In [ ]:
# set initial
# options = ['A','B','C','D']

options = ['A','B','C','D']
toggle.param.set_param(options=dict(zip(options,options)), value=['D'])

In [ ]:
test_hs_pf_mixin = HideShowPlacefieldsRenderingMixin(name='ASCII Sine Wave')
pn.Row(test_hs_pf_mixin.param)

In [ ]:
class Sine(param.Parameterized):

    phase = param.Number(default=0, bounds=(0, np.pi))

    frequency = param.Number(default=1, bounds=(0.1, 2))

    @param.depends('phase', 'frequency')
    def view(self):
        y = np.sin(np.linspace(0, np.pi * 3, 40) * self.frequency + self.phase)
        y = ((y - y.min()) / y.ptp()) * 20
        array = np.array(
            [list((' ' * (int(round(d)) - 1) + '*').ljust(20)) for d in y])
        return pn.pane.Str('\n'.join([''.join(r) for r in array.T]), height=380, width=500)


sine = Sine(name='ASCII Sine Wave')
pn.Row(sine.param, sine.view)